## FEFE DATA EXLORATION

#### 1. FIX DATASET -> ASSOCIATE COMPONENTS WITH BODE CODE

In [1]:
# 1 - initial dataframe exploration from chris
import pandas as pd
datapath = "../data/raw/componentwise_dataset.xlsx"
df = pd.read_excel(datapath, sheet_name="Export")

cols_en = [
"Fault start date (MEL)",
"Short text (MEL)",
"Long text (MEL)",
"Message number (MEL)",
"Vehicle family (TPL)",
"Vehicle type (TPL)",
"Material (CO)",
"Material reference (MAT)",
"Material quantity ACTUAL",
"Material / unit of measure (CO)",
"WF screen symptom (MEL)",
"WF-cause-sys-1 (MEL) (H)",
"WF-cause-sys-2 (MEL) (H)",
"WF-Cause-Sys-3 (MEL) (H)",
"WF-Cause-Sys-4 (MEL) (H)",
"WF-Cause-Sys-5 (MEL) (H)",
"WF-cause-sys-6 (MEL) (H)",
"WF cause failure type (MEL)",
"WF-cause-fix.mass (1-3) (MEL)",
"Multiple unit / EW (TPL)",
"Functional location (TPL)",
"DIN code (H)",
"Design workstation (location) (MEL)"
]

cols_interst = [
"Fault start date (MEL)",
"Short text (MEL)",
"Long text (MEL)",
"Message number (MEL)",
"Material reference (MAT)",
"WF cause failure type (MEL)",
"WF-cause-fix.mass (1-3) (MEL)",
"Multiple unit / EW (TPL)",

]
df = df.loc[:1594]

df.columns = cols_en
df = df[cols_interst]

df.rename(columns={"Message number (MEL)" : "log_number",
                   "Multiple unit / EW (TPL)": "train_number",
                   "Fault start date (MEL)" : "fault_start_date",
                   "Short text (MEL)" : "problem",
                   "Long text (MEL)" : "maintenance_log",
                   "Material reference (MAT)": "material_description",
                   "WF cause failure type (MEL)": "cause_of_failure",
                   "WF-cause-fix.mass (1-3) (MEL)": "maintenance_steps",
                   "Material reference (MAT)": "material_name"}
                   , inplace=True)

df["fault_start_date"] = pd.to_datetime(df["fault_start_date"], format="%Y-%m-%d %H:%M:%S")
df["problem"] = df["problem"].astype(str)
df["maintenance_log"] = df["maintenance_log"].astype(str)
df["log_number"] = df["log_number"].astype(int)
df["train_number"] = df["train_number"].astype(str)
df["material_name"] = df["material_name"].astype(str)
df["maintenance_steps"] = df["maintenance_steps"].astype(str)
df["cause_of_failure"] = df["cause_of_failure"].astype(str)

# Ok it is always the same. Remove the prefix:
df["train_number"]= df["train_number"].str[10:]

# Reorder columns
df = df[["train_number", "problem", "log_number", "maintenance_log", "fault_start_date", 
         "material_name", "maintenance_steps", "cause_of_failure"]]

In [2]:
# 2.2 - associate components to bode code
components_bode_dict = {
    'SICHERHEITSGLAS': '25-351-0224-301',
    'ZAHNRIEMENRAD': '25-865-0011-301',
    'DREHFALLE': '25-341-0119-101',
    'TÜRSTEUERUNG': '25-004-1019-301',
    'FÜHRUNGSSCHIENE': '25-326-0300-201',
    'TÜRFLÜGEL': '25-310-0566-202'
    

}

# keep only rows that have one of these components
def search_material_reference(row, search_dict):
    for key in search_dict.keys():
        if key.lower() in row['material_name'].lower() or key.lower() in row['problem'].lower():
            return True
    return False

filtered_df = df[df.apply(lambda row: search_material_reference(row, components_bode_dict), axis=1)]
filtered_df = filtered_df.drop_duplicates()

# add bode code
def add_bode_code(row, search_dict):
    for key, code in search_dict.items():
        if key.lower() in row['material_name'].lower() or key.lower() in row['problem'].lower():
            return code
    return None

filtered_df['BODE code'] = filtered_df.apply(lambda row: add_bode_code(row, components_bode_dict), axis=1)
print(filtered_df)

     train_number                                 problem  log_number  \
8           015-5                Primärdrehfalle tauschen   131479425   
9           001-5  -ET-Antriebsmotor 1 gestört-zeitwei...   131471966   
13          047-8                -ET-Rechner 3 Stör.Aktiv   131461040   
15          043-7            -ÜT-Türrechner WE1 abgenutzt   131460609   
46          023-9           -ET-Rechner 2 schliesst nicht   131351602   
...           ...                                     ...         ...   
1557        024-7                  ET-Verriegelung 3 lose   128275990   
1567        046-0     ET-v-Schwelle Notent 3 schwergängig   128189630   
1569        056-9             -Türsteuerung Zug 2 pendelt   128140205   
1570        056-9             -Türsteuerung Zug 2 pendelt   128140205   
1594        009-8                ET-Türblatt L korrodiert   127402644   

                                        maintenance_log fault_start_date  \
8      * 18.09.2024 12:56:19 CET (U239359) * Sy

In [3]:
# 3 - save df
filtered_df.to_csv("../data/interim/final_df.csv", index=False)

In [4]:
filtered_df.head()

,train_number,problem,log_number,maintenance_log,fault_start_date,material_name,maintenance_steps,cause_of_failure,BODE code
8,015-5,Primärdrehfalle tauschen,131479425,* 18.09.2024 12:56:19 CET (U239359) * System ...,2024-09-18,Drehfalle vollständig zu Einstiegstüre,Sonstige → Test / Simulation durchgeführt,Software & Firmware → übrige Störungen,25-341-0119-101
9,001-5,-ET-Antriebsmotor 1 gestört-zeitwei...,131471966,* 17.09.2024 09:18:21 CET (U147283) Tel. +41 ...,2024-09-17,Türsteuerung BODE 25-004-1019-301,HW/Elektr. & Mech. → ersetzt,HW/Elektr. & Mech. - elektrisch → spricht nich...,25-004-1019-301
13,047-8,-ET-Rechner 3 Stör.Aktiv,131461040,* 14.09.2024 06:23:11 CET (U224474) Tel. +41 ...,2024-09-14,Drehfalle vollständig zu Einstiegstüre,HW/Elektr. & Mech. → ersetzt ; HW/Elektr. & Me...,HW/Elektr. & Mech. - elektrisch → Fehler Daten...,25-341-0119-101
15,043-7,-ÜT-Türrechner WE1 abgenutzt,131460609,* 13.09.2024 21:30:31 CET (U146319) Tel. +41 5...,2024-09-13,Türsteuerung MTS-L 1.14 zu Uebergangstür,HW/Elektr. & Mech. → ersetzt ; Software → Upda...,Software & Firmware → übrige Störungen,25-004-1019-301
46,023-9,-ET-Rechner 2 schliesst nicht,131351602,* 18.08.2024 13:41:57 CET (U230689) Tel. +41 ...,2024-08-18,Handschalter zu Türflügel,HW/Elektr. & Mech. → ersetzt,HW/Elektr. & Mech. - elektrisch → spricht unbe...,25-310-0566-202


#### 2. WORK ON MAINTENANCE LOG COLUMN

In [5]:
import pandas as pd
import json
import re

df = filtered_df
# extract log
def extract_logs(maintenance_log):
    log_pattern = r'(\d{2}\.\d{2}\.\d{4} \d{2}:\d{2}:\d{2}) CET \((U\d+)\).*?(\*.*?)(?=(\d{2}\.\d{2}\.\d{4} \d{2}:\d{2}:\d{2}) CET|\Z)'
    matches = re.findall(log_pattern, maintenance_log, re.DOTALL)
    logs = []
    for match in matches:
        log = {
            "date_time": match[0],
            "comment": match[2].strip(),
        }
        logs.append(log)
    
    return logs

log_dict = {}
for index, row in df.iterrows():
    logs = extract_logs(row['maintenance_log'])
    print(logs)
    for i, log in enumerate(logs):
        key = f"issue_{int(row['log_number'])}_log_{i+1}"
        log_dict[key] = log


[{'date_time': '18.09.2024 12:56:19', 'comment': '* System (Störungsursache): * Einstiegssysteme --> Einstiegstüre --> Türmechanik --> Ver-/ * Entriegelungseinheit * * Lokalisierung (Störungsursache): * Türe 3 * * Symptom (Störungsbild): * lose * *'}, {'date_time': '18.09.2024 12:55:02', 'comment': '*'}, {'date_time': '18.09.2024 12:54:54', 'comment': '* System (Störungsursache): * Einstiegssysteme --> Einstiegstüre --> Türmechanik --> Ver-/ * Entriegelungseinheit * * Lokalisierung (Störungsursache): * Türe 1 * * Symptom (Störungsbild): * lose * *'}, {'date_time': '18.09.2024 12:54:39', 'comment': '* Beim Kontrollauftrag/Änderungsauftrag Bolzenmarkierung wurde an der MT23   eine Drehfalle mit gebrochenem Siegellack an der Mutter des Bolzens   gefunden. Die Mutter des Bolzens war lose. * Mutter wurde wieder angezogen. * Die Primärdrehfalle 94-000-1950A muss beim nächsten UH ersetzt werden *'}, {'date_time': '20.09.2024 16:11:04', 'comment': '* Ausfallart (Störungsursache): * übrige Stör

In [6]:
# Extract the door number from the maintenance log and put it as a column
def extract_door_number(text):
    pattern = r'(?:Localisation \(Cause\)|Lokalisierung \(Störungsursache\)):\s*\*?\s*(?:Türe|Porte)?\s*(\d+)\s*\*?'
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return None

df["door"] = df["maintenance_log"].apply(extract_door_number)
df.dropna(subset=['door'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [7]:
df['maintenance_log'] = df['log_number'].map(lambda log_number: [f"issue_{int(log_number)}_log_{i+1}" for i in range(len(extract_logs(df.loc[df['log_number'] == log_number, 'maintenance_log'].values[0])))])
print(df)

print(json.dumps(log_dict, indent=4))

    train_number                                 problem  log_number  \
0          015-5                Primärdrehfalle tauschen   131479425   
1          001-5  -ET-Antriebsmotor 1 gestört-zeitwei...   131471966   
2          047-8                -ET-Rechner 3 Stör.Aktiv   131461040   
3          023-9           -ET-Rechner 2 schliesst nicht   131351602   
4          113-8                     +ÜT moteur 5 bloqué   131332603   
..           ...                                     ...         ...   
153        120-3           ET verrouillage 4 AAZ12578936   128404198   
154        024-7                  ET-Verriegelung 3 lose   128275990   
155        046-0     ET-v-Schwelle Notent 3 schwergängig   128189630   
156        056-9             -Türsteuerung Zug 2 pendelt   128140205   
157        056-9             -Türsteuerung Zug 2 pendelt   128140205   

                                       maintenance_log fault_start_date  \
0    [issue_131479425_log_1, issue_131479425_log_2,...      

In [8]:
#save df
df.to_csv("../data/interim/final_df2.csv", index=False)

In [9]:
# save json
json_file_path = '../data/interim/maintenance_logs.json'
with open(json_file_path, 'w',encoding='utf-8') as json_file:
    json.dump(log_dict, json_file, indent=4, ensure_ascii=False)

### 3 - CREATE JSON FOR MENU

####  3.1 - come dare priorità

In [10]:
# 3.1.1 - divido equamente i fault start date in 3 parti (si puo rivedere questo)
date_edges = [df["fault_start_date"].quantile(0.33), df["fault_start_date"].quantile(0.66)]
def assign_priority(date, edges):
    if date <= edges[0]:
        return 1
    elif date <= edges[1]:
        return 2
    else:
        return 3
# 3.1.2 - aggiungo la colonna 'priority'
df['priority'] = df['fault_start_date'].apply(assign_priority, edges=date_edges)


In [11]:
df.head(10)

,train_number,problem,log_number,maintenance_log,fault_start_date,material_name,maintenance_steps,cause_of_failure,BODE code,door,priority
0,015-5,Primärdrehfalle tauschen,131479425,"[issue_131479425_log_1, issue_131479425_log_2,...",2024-09-18,Drehfalle vollständig zu Einstiegstüre,Sonstige → Test / Simulation durchgeführt,Software & Firmware → übrige Störungen,25-341-0119-101,3,3
1,001-5,-ET-Antriebsmotor 1 gestört-zeitwei...,131471966,"[issue_131471966_log_1, issue_131471966_log_2,...",2024-09-17,Türsteuerung BODE 25-004-1019-301,HW/Elektr. & Mech. → ersetzt,HW/Elektr. & Mech. - elektrisch → spricht nich...,25-004-1019-301,1,3
2,047-8,-ET-Rechner 3 Stör.Aktiv,131461040,"[issue_131461040_log_1, issue_131461040_log_2,...",2024-09-14,Drehfalle vollständig zu Einstiegstüre,HW/Elektr. & Mech. → ersetzt ; HW/Elektr. & Me...,HW/Elektr. & Mech. - elektrisch → Fehler Daten...,25-341-0119-101,3,3
3,023-9,-ET-Rechner 2 schliesst nicht,131351602,"[issue_131351602_log_1, issue_131351602_log_2,...",2024-08-18,Handschalter zu Türflügel,HW/Elektr. & Mech. → ersetzt,HW/Elektr. & Mech. - elektrisch → spricht unbe...,25-310-0566-202,2,3
4,113-8,+ÜT moteur 5 bloqué,131332603,"[issue_131332603_log_1, issue_131332603_log_2,...",2024-08-14,Türsteuerung MTS-L 1.14 zu Uebergangstür,HW/Elektr. & Mech. → ersetzt,Software & Firmware → übrige Störungen,25-004-1019-301,5,3
5,058-5,ET 1 gestört-zeitweise,131304580,"[issue_131304580_log_1, issue_131304580_log_2,...",2024-08-08,Drehfalle vollständig zu Einstiegstüre,Sonstige → Test / Simulation durchgeführt,HW/Elektr. & Mech. - mechanisch → fehlt,25-341-0119-101,1,3
6,017-1,-ET 1 n'ouvre pas,131280057,"[issue_131280057_log_1, issue_131280057_log_2,...",2024-08-01,Türsteuerung BODE 25-004-1019-301,HW/Elektr. & Mech. → ersetzt,HW/Elektr. & Mech. - elektrisch → spricht nich...,25-004-1019-301,1,3
7,015-5,-ET 2 schliesst nicht,131252085,"[issue_131252085_log_1, issue_131252085_log_2,...",2024-07-25,Drehfalle vollständig zu Einstiegstüre,HW/Elektr. & Mech. → ersetzt ; Sonstige → gepr...,HW/Elektr. & Mech. - mechanisch → abgebrochen ...,25-341-0119-101,2,3
8,048-6,-Schiebetritt Antrieb 2 fährt nicht...,131243142,"[issue_131243142_log_1, issue_131243142_log_2,...",2024-07-23,Drehfalle vollständig zu Einstiegstüre,HW/Elektr. & Mech. → Messwert(e) ausgelesen / ...,HW/Elektr. & Mech. - elektrisch → startet nich...,25-341-0119-101,2,3
9,030-4,-ET calculateur porte 4 déclenché,131198042,"[issue_131198042_log_1, issue_131198042_log_2,...",2024-07-11,Türsteuerung BODE 25-004-1019-301,Software → Update / Upgrade aufgespielt,HW/Elektr. & Mech. - elektrisch → startet nich...,25-004-1019-301,4,3


#### 3.2 - crea JSON per il menu maintenance

In [15]:
# 3.2.1 - file json per il menu maintenance = priority, train number, door number, problem
mm_dict = {}
s = []
for i,row in df.iterrows():
    log = {
            'priority': row['priority'],
            'train_number': row['train_number'],
            'door_number': row['door'],
            'problem': row['problem']
        }
    s.append(log)
mm_dict['tasks'] = s
# save json
json_file_path = '../data/interim/maintenance_menu.json'
with open(json_file_path, 'w',encoding='utf-8') as json_file:
    json.dump(mm_dict, json_file, indent=4, ensure_ascii=False)

#### 3.3 - crea JSON per il menu steps

In [13]:
# 3.3.1 - file json per il menu steps = step number, step description, component code
steps = ['Close the door leaf.', 'On the inside of the left door leaf, remove 2 hexagonal socket screws from the cover of the door leaf distributor (SW 4).','Remove the cover of the distribution box.']
steps_BODE=['25-326-0300-201','25-714-6068-321','25-950-0002-309']
ms_dict = {}
s = []
for i,el in enumerate(steps):
    log = {
            'stepN': i,
            'step description': el,
            'component code': steps_BODE[i]
        }
    s.append(log)
ms_dict['name'] = '6.7.6_Türflügel_ausbauen'
ms_dict['steps'] = s
# save json
print(s)
json_file_path = '../data/interim/maintenance_steps_676.json'
with open(json_file_path, 'w') as json_file:
    json.dump(ms_dict, json_file, indent=4)

[{'stepN': 0, 'step description': 'Close the door leaf.', 'component code': '25-326-0300-201'}, {'stepN': 1, 'step description': 'On the inside of the left door leaf, remove 2 hexagonal socket screws from the cover of the door leaf distributor (SW 4).', 'component code': '25-714-6068-321'}, {'stepN': 2, 'step description': 'Remove the cover of the distribution box.', 'component code': '25-950-0002-309'}]


In [14]:
# FATTO
{
    'priority': '1-5',
    'train number': '2222',
    'door number': '3',
    'problem': '+ÜT moteur 5 bloqué'
}

# questo ce lo abbiamo gia no?
{
    'history': [
        {
            'date': '2021-01-01',
            'comment': 'Issue identified'
        },
        {
            'date': '2021-01-02',
            'comment': 'Issue fixed'
        }
    ],

    'how': 'avvia manuentizione'

}


{
    'step1': {'open the outer box', 'pezzo che si illumina', 'next button'},
    'step2': {'bla bla bla ', 'pezzo che si illumina', 'next button'},
    'step3': {'bla bla bla bla', 'pezzo che si illumina', 'next button'}
}

# After having concluded the maintenance, the user can close the issue and the issue will be removed from the list of issues


{'step1': {'next button', 'open the outer box', 'pezzo che si illumina'},
 'step2': {'bla bla bla ', 'next button', 'pezzo che si illumina'},
 'step3': {'bla bla bla bla', 'next button', 'pezzo che si illumina'}}